In [7]:
import requests;


In [9]:
with open(r"C:\Users\Saksham\Desktop\CC_task\pokemon.txt","r") as pk:
    content = pk.readlines()
    print(content)

['Amoonguss\n', 'Gyarados\n', '143\n', 'Volcarona\n', 'Wooper\n', 'Greninja\n', '231\n', 'Charizard\n', 'Sceptile\n', '681\n', 'Blaziken\n', 'Mudkip\n', 'Rowlet\n', '292\n', 'Wynaut\n', 'Incineroar\n', 'Spheal\n', 'Buzzwole\n', 'Salamence\n', '778\n', 'Metagross\n', 'Rayquaza\n', 'Falinks\n', 'Infernape\n', 'Piplup\n', 'Dragapult\n', 'Darkrai\n', 'Gholdengo\n', '501\n', 'Walking-Wake\n', 'Milotic\n', 'Bulbasaur\n', 'Gengar\n', 'Umbreon\n', 'Garchomp\n', 'Chandelure\n', 'Lucario\n', 'Flygon\n', 'Scizor\n', 'Squirtle\n', 'Dragonite\n', 'Luxray\n', 'Lugia\n', 'Noivern\n', 'Ceruledge\n', 'Haxorus\n', 'Psyduck\n', 'Shedinja\n', 'Annihilape\n', 'Magikarp']


In [51]:
len(content)

50

In [10]:
my_dict={}

In [12]:
for i in range(0,50): #50 is the length
    name_id = content[i].strip()
    r1=requests.get(f"https://pokeapi.co/api/v2/pokemon/{name_id}/")
    name=r1.json()["name"]
    idno=r1.json()["id"]
    type_=[]
    abilities=[]
    for ability in r1.json()["abilities"]:
            abilities.append(ability['ability']['name'])
    for types in r1.json()["types"]:
            type_.append(types['type']['name'])
    r2=requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{name_id}/")
    legendary=r2.json()["is_legendary"]
    mythical=r2.json()["is_mythical"]
    temp_dict={"id":idno,"is_legendary":legendary,"is_mythical":mythical,"abilities":abilities,"types":type_}
    my_dict[name]=temp_dict

In [33]:
damage_array={
    "attackers": {},
    "defenders": {}
}
pokemon_types = [
    "normal",
    "fire",
    "water",
    "electric",
    "grass",
    "ice",
    "fighting",
    "poison",
    "ground",
    "flying",
    "psychic",
    "bug",
    "rock",
    "ghost",
    "dragon",
    "dark",
    "steel",
    "fairy"
]
for j in range(1,19):
    r3=requests.get(f"https://pokeapi.co/api/v2/type/{j}/")
    name=r3.json()['name']
    for k in (r3.json()['damage_relations']):
            temp_attacker={}
            temp_defender={}
            for ele1 in r3.json()['damage_relations']["double_damage_from"]:
                        temp_defender[ele1["name"]]=2
            for ele1 in r3.json()['damage_relations']["double_damage_to"]:
                        temp_attacker[ele1["name"]]=2
            for ele1 in r3.json()['damage_relations']["half_damage_from"]:
                        temp_defender[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["half_damage_to"]:
                        temp_attacker[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["no_damage_from"]:
                        temp_defender[ele1["name"]]=0
            for ele1 in r3.json()['damage_relations']["no_damage_to"]:
                        temp_attacker[ele1["name"]]=0
    if (temp_attacker):    
        for ele in pokemon_types:
                    if ele not in temp_attacker.keys():
                            temp_attacker[ele]=1
        damage_array["attackers"].update({name:temp_attacker})
    if (temp_defender):
            for ele in pokemon_types:
                    if ele not in temp_defender.keys():
                            temp_defender[ele]=1
            damage_array["defenders"].update({name:temp_defender})
damage_array


{'attackers': {'normal': {'rock': 0.5,
   'steel': 0.5,
   'ghost': 0,
   'normal': 1,
   'fire': 1,
   'water': 1,
   'electric': 1,
   'grass': 1,
   'ice': 1,
   'fighting': 1,
   'poison': 1,
   'ground': 1,
   'flying': 1,
   'psychic': 1,
   'bug': 1,
   'dragon': 1,
   'dark': 1,
   'fairy': 1},
  'fighting': {'normal': 2,
   'rock': 2,
   'steel': 2,
   'ice': 2,
   'dark': 2,
   'flying': 0.5,
   'poison': 0.5,
   'bug': 0.5,
   'psychic': 0.5,
   'fairy': 0.5,
   'ghost': 0,
   'fire': 1,
   'water': 1,
   'electric': 1,
   'grass': 1,
   'fighting': 1,
   'ground': 1,
   'dragon': 1},
  'flying': {'fighting': 2,
   'bug': 2,
   'grass': 2,
   'rock': 0.5,
   'steel': 0.5,
   'electric': 0.5,
   'normal': 1,
   'fire': 1,
   'water': 1,
   'ice': 1,
   'poison': 1,
   'ground': 1,
   'flying': 1,
   'psychic': 1,
   'ghost': 1,
   'dragon': 1,
   'dark': 1,
   'fairy': 1},
  'poison': {'grass': 2,
   'fairy': 2,
   'poison': 0.5,
   'ground': 0.5,
   'rock': 0.5,
   'ghost': 

In [34]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route("/")
def damage_endpoint():
    attacker = request.args.get("attacker")
    defender = request.args.get("defender")
    
    # Check if both are missing
    if not attacker and not defender:
        return jsonify({"error": "Provide attacker and defender"}), 400
    
    # Only attacker missing
    elif not attacker:
        try:
            return jsonify(damage_array["defenders"][defender])
        except KeyError:
            return jsonify({"error": f"No data for defender {defender}"}), 404
    
    # Only defender missing
    elif not defender:
        try:
            return jsonify(damage_array["attackers"][attacker])
        except KeyError:
            return jsonify({"error": f"No data for attacker {attacker}"}), 404
    
    # Both present
    else:
        try:
            result = damage_array["attackers"][attacker][defender]
            return jsonify({"attacker": attacker, "defender": defender, "multiplier": result})
        except KeyError:
            return jsonify({"error": f"No data for {attacker} vs {defender}"}), 404



In [ ]:
from threading import Thread
def run_app():
    app.run(port=5000)

thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
